In [ ]:
ls /usr/local/lib/python3.11/dist-packages | grep cusparse

cusparselt/
nvidia_cusparse_cu12-12.3.1.170.dist-info/
nvidia_cusparselt_cu12-0.6.2.dist-info/
~vidia_cusparse_cu12-12.5.1.3.dist-info/


In [ ]:
rm -rf /usr/local/lib/python3.11/dist-packages/~vidia*

In [ ]:
pip cache purge

Files removed: 72


In [ ]:
!pip install ultralytics

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import zipfile
import os

with zipfile.ZipFile("mask_project.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/mask_project")

In [ ]:
import shutil
import random
from pathlib import Path

# Yolunuzu təyin edin
root = '/content/mask_project'
image_dir = os.path.join(root, 'images')
label_dir = os.path.join(root, 'labels')

# Yeni alt qovluqlar (train / val)
for split in ['train', 'val']:
    os.makedirs(os.path.join(image_dir, split), exist_ok=True)
    os.makedirs(os.path.join(label_dir, split), exist_ok=True)

# Bütün .jpg faylları götür
all_images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
random.shuffle(all_images)

# 80% train, 20% val
split_index = int(0.8 * len(all_images))
train_files = all_images[:split_index]
val_files = all_images[split_index:]

# Faylları böl
def move_files(file_list, split):
    for file in file_list:
        image_src = os.path.join(image_dir, file)
        label_src = os.path.join(label_dir, file.replace('.jpg', '.txt'))

        image_dst = os.path.join(image_dir, split, file)
        label_dst = os.path.join(label_dir, split, file.replace('.jpg', '.txt'))

        shutil.move(image_src, image_dst)
        shutil.move(label_src, label_dst)

move_files(train_files, 'train')
move_files(val_files, 'val')

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')
model.train(data='data.yaml', epochs=5, imgsz=640)

Ultralytics 8.3.110 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torch

FileNotFoundError: [34m[1mtrain: [0mError loading data from /content/mask_project/images/train
See https://docs.ultralytics.com/datasets for dataset formatting guidance.

In [ ]:
python train_mask_model.py
